# 1. 컨볼루션 적용

## 참고자료

https://victorzhou.com/blog/keras-cnn-tutorial/

In [1]:
import keras
import numpy as np

Using TensorFlow backend.


## 데이터 가져오기

In [14]:
import mnist


# The first time you run this might be a bit slow, since the
# mnist package has to download and cache the data.
train_images = mnist.train_images()
train_labels = mnist.train_labels()

print(train_images.shape) # (60000, 28, 28)
print(train_labels.shape) # (60000,)

(60000, 28, 28)
(60000,)


In [15]:
dir(mnist)

['IdxDecodeError',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__version__',
 'array',
 'datasets_url',
 'download_and_parse_mnist_file',
 'download_file',
 'functools',
 'gzip',
 'numpy',
 'operator',
 'os',
 'parse_idx',
 'struct',
 'tempfile',
 'temporary_dir',
 'test_images',
 'test_labels',
 'train_images',
 'train_labels',
 'urljoin',
 'urlretrieve']

In [16]:
test_images = mnist.test_images()
test_labels = mnist.test_labels()

In [17]:
# Normalize the images.
train_images = (train_images / 255) - 0.5
test_images = (test_images / 255) - 0.5

# Reshape the images.
train_images = np.expand_dims(train_images, axis=3)
test_images = np.expand_dims(test_images, axis=3)

print(train_images.shape) # (60000, 784)
print(test_images.shape)  # (10000, 784)

(60000, 28, 28, 1)
(10000, 28, 28, 1)


In [18]:
from keras.utils import to_categorical

In [19]:
train_labels = to_categorical(train_labels)

In [20]:
test_labels = to_categorical(test_labels)

In [21]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten


## 모델과 레이어 지정

In [26]:

num_filters = 8
filter_size = 3
pool_size = 2

model = Sequential([
  Conv2D(num_filters, filter_size, input_shape=(28, 28, 1)),
  MaxPooling2D(pool_size=pool_size),
  Flatten(),
  Dense(10, activation='softmax'),
])

## 컴파일

In [27]:
model.compile(
  optimizer='adam',
  loss='categorical_crossentropy',
  metrics=['accuracy'],
)

## 훈련하기

In [28]:
model.fit(
  train_images, # training data
  train_labels, # training targets
  epochs=5,
  batch_size=32,
  validation_split= 0.2
)

Train on 48000 samples, validate on 12000 samples
Epoch 1/5
48000/48000 [==============================] - 5s 104us/step - loss: 0.3676 - accuracy: 0.8930 - val_loss: 0.2101 - val_accuracy: 0.9392
Epoch 2/5
48000/48000 [==============================] - 5s 102us/step - loss: 0.1899 - accuracy: 0.9444 - val_loss: 0.1521 - val_accuracy: 0.9568
Epoch 3/5
48000/48000 [==============================] - 5s 102us/step - loss: 0.1453 - accuracy: 0.9576 - val_loss: 0.1289 - val_accuracy: 0.9631
Epoch 4/5
48000/48000 [==============================] - 5s 99us/step - loss: 0.1197 - accuracy: 0.9658 - val_loss: 0.1102 - val_accuracy: 0.9688
Epoch 5/5
48000/48000 [==============================] - 5s 99us/step - loss: 0.1054 - accuracy: 0.9689 - val_loss: 0.1046 - val_accuracy: 0.9711


##  평가하기

In [29]:
model.evaluate(
  test_images,
  test_labels
)

10000/10000 [==============================] - 0s 38us/step


[0.10238509910665453, 0.9686999917030334]

## 예측하기

In [30]:
# Predict on the first 5 test images.
predictions = model.predict(test_images[:5])

# Print our model's predictions.
print(np.argmax(predictions, axis=1)) # [7, 2, 1, 0, 4]


[7 2 1 0 4]


In [31]:
# Check our predictions against the ground truths.
print(test_labels[:5]) # [7, 2, 1, 0, 4]

[[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]]


# 2. 모델 다시 로딩

## 기존 처리 저장

In [32]:
model.save_weights('cnn.h5')

In [33]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten

num_filters = 8
filter_size = 3
pool_size = 2

# Build the model.
model_ = Sequential([
  Conv2D(num_filters, filter_size, input_shape=(28, 28, 1)),
  MaxPooling2D(pool_size=pool_size),
  Flatten(),
  Dense(10, activation='softmax'),
])

# Load the model's saved weights.
model_.load_weights('cnn.h5')

In [34]:
# Predict on the first 5 test images.
predictions = model_.predict(test_images[:5])

# Print our model's predictions.
print(np.argmax(predictions, axis=1)) # [7, 2, 1, 0, 4]

# Check our predictions against the ground truths.
print(test_labels[:5]) # [7, 2, 1, 0, 4]

[7 2 1 0 4]
[[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]]


# 3. 확장

성능을 향상 시키기 위한 조정 방법

## 컨볼루션 계층 추가

In [35]:
model = Sequential([
  Conv2D(num_filters, filter_size, input_shape=(28, 28, 1)),
  Conv2D(num_filters, filter_size),
  MaxPooling2D(pool_size=pool_size),
  Flatten(),
  Dense(10, activation='softmax'),
])

In [37]:
model.compile(
  optimizer='adam',
  loss='categorical_crossentropy',
  metrics=['accuracy'],
)

In [38]:
model.fit(
  train_images, # training data
  train_labels, # training targets
  epochs=5,
  batch_size=32,
  validation_split= 0.2
)

Train on 48000 samples, validate on 12000 samples
Epoch 1/5
48000/48000 [==============================] - 10s 208us/step - loss: 0.3112 - accuracy: 0.9074 - val_loss: 0.1434 - val_accuracy: 0.9582
Epoch 2/5
48000/48000 [==============================] - 10s 208us/step - loss: 0.1259 - accuracy: 0.9629 - val_loss: 0.1018 - val_accuracy: 0.9712
Epoch 3/5
48000/48000 [==============================] - 10s 209us/step - loss: 0.0925 - accuracy: 0.9718 - val_loss: 0.0945 - val_accuracy: 0.9717
Epoch 4/5
48000/48000 [==============================] - 10s 210us/step - loss: 0.0768 - accuracy: 0.9758 - val_loss: 0.0842 - val_accuracy: 0.9753
Epoch 5/5
48000/48000 [==============================] - 10s 208us/step - loss: 0.0646 - accuracy: 0.9804 - val_loss: 0.0826 - val_accuracy: 0.9770


## 드롭 아웃 추가

In [39]:
from keras.layers import Dropout

model = Sequential([
  Conv2D(num_filters, filter_size, input_shape=(28, 28, 1)),
  MaxPooling2D(pool_size=pool_size),
  Dropout(0.5),
  Flatten(),
  Dense(10, activation='softmax'),
])

In [40]:
model.compile(
  optimizer='adam',
  loss='categorical_crossentropy',
  metrics=['accuracy'],
)

In [41]:
model.fit(
  train_images, # training data
  train_labels, # training targets
  epochs=5,
  batch_size=32,
  validation_split= 0.2
)

Train on 48000 samples, validate on 12000 samples
Epoch 1/5
48000/48000 [==============================] - 5s 113us/step - loss: 0.5058 - accuracy: 0.8447 - val_loss: 0.2651 - val_accuracy: 0.9243
Epoch 2/5
48000/48000 [==============================] - 6s 115us/step - loss: 0.3150 - accuracy: 0.9056 - val_loss: 0.1994 - val_accuracy: 0.9456
Epoch 3/5
48000/48000 [==============================] - 5s 111us/step - loss: 0.2707 - accuracy: 0.9184 - val_loss: 0.1698 - val_accuracy: 0.9546
Epoch 4/5
48000/48000 [==============================] - 5s 110us/step - loss: 0.2442 - accuracy: 0.9265 - val_loss: 0.1535 - val_accuracy: 0.9579
Epoch 5/5
48000/48000 [==============================] - 5s 112us/step - loss: 0.2332 - accuracy: 0.9274 - val_loss: 0.1445 - val_accuracy: 0.9592


## 완전 연결 계층 추가

In [42]:
model = Sequential([
  Conv2D(num_filters, filter_size, input_shape=(28, 28, 1)),
  MaxPooling2D(pool_size=pool_size),
  Flatten(),
  Dense(64, activation='relu'),
  Dense(10, activation='softmax'),
])

In [43]:
model.compile(
  optimizer='adam',
  loss='categorical_crossentropy',
  metrics=['accuracy'],
)

In [44]:
model.fit(
  train_images, # training data
  train_labels, # training targets
  epochs=5,
  batch_size=32,
  validation_split= 0.2
)

Train on 48000 samples, validate on 12000 samples
Epoch 1/5
48000/48000 [==============================] - 6s 115us/step - loss: 0.3067 - accuracy: 0.9099 - val_loss: 0.1438 - val_accuracy: 0.9577
Epoch 2/5
48000/48000 [==============================] - 5s 111us/step - loss: 0.1227 - accuracy: 0.9636 - val_loss: 0.1064 - val_accuracy: 0.9688
Epoch 3/5
48000/48000 [==============================] - 5s 108us/step - loss: 0.0840 - accuracy: 0.9743 - val_loss: 0.0880 - val_accuracy: 0.9736
Epoch 4/5
48000/48000 [==============================] - 5s 110us/step - loss: 0.0624 - accuracy: 0.9803 - val_loss: 0.0804 - val_accuracy: 0.9762
Epoch 5/5
48000/48000 [==============================] - 5s 109us/step - loss: 0.0487 - accuracy: 0.9849 - val_loss: 0.0761 - val_accuracy: 0.9773


## 컨볼류션 파라미터 조정

In [45]:
num_filters = 8
filter_size = 3

model = Sequential([
  # See https://keras.io/layers/convolutional/#conv2d for more info.
  Conv2D(
    num_filters,
    filter_size,
    input_shape=(28, 28, 1),
    strides=2,
    padding='same',
    activation='relu',
  ),
  MaxPooling2D(pool_size=pool_size),
  Flatten(),
  Dense(10, activation='softmax'),
])

In [46]:
model.compile(
  optimizer='adam',
  loss='categorical_crossentropy',
  metrics=['accuracy'],
)

In [47]:
model.fit(
  train_images, # training data
  train_labels, # training targets
  epochs=5,
  batch_size=32,
  validation_split= 0.2
)

Train on 48000 samples, validate on 12000 samples
Epoch 1/5
48000/48000 [==============================] - 3s 68us/step - loss: 0.5267 - accuracy: 0.8540 - val_loss: 0.2224 - val_accuracy: 0.9365
Epoch 2/5
48000/48000 [==============================] - 3s 68us/step - loss: 0.2033 - accuracy: 0.9405 - val_loss: 0.1675 - val_accuracy: 0.9503
Epoch 3/5
48000/48000 [==============================] - 3s 67us/step - loss: 0.1598 - accuracy: 0.9523 - val_loss: 0.1492 - val_accuracy: 0.9567
Epoch 4/5
48000/48000 [==============================] - 3s 65us/step - loss: 0.1376 - accuracy: 0.9583 - val_loss: 0.1363 - val_accuracy: 0.9592
Epoch 5/5
48000/48000 [==============================] - 3s 65us/step - loss: 0.1229 - accuracy: 0.9630 - val_loss: 0.1200 - val_accuracy: 0.9633
